# Day 17

## Part 1

- rocks fall! ...everyone hopefuly doesn't die
- rocks fall in the same order: `-`, `+`, `J`, `|`, `o`
- the rocks don't spin but can translate left and right
- this is all in a vertical chamber that is 7 units wide
- rocks appear such that it's left edge is 2 units from the left wall and it's bottom is 3 units above the highest rock in the room/the floor.
- rocks alternate between being pushed by hot gas and falling
- if a rock would move into a wall or the floor or another rock it doesn't
- a new rock appears immediately after the last rock stops being able to fall further

`How many units tall will the tower of rocks be after 2022 rocks have stopped falling?`